In [55]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria,
    BeamSearchScorer,
)
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer.pad_token_id = tokenizer.eos_token_id
stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=30)])

# prompt = "Today I believe we can finally"
prompt = "It might be possible to"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# outputs = model.generate(input_ids, do_sample=False, max_length = 30)
# tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [56]:
#Greedy Search

model.config.pad_token_id = model.config.eos_token_id
model.generation_config.pad_token_id = model.generation_config.eos_token_id
logits_processor = LogitsProcessorList([MinLengthLogitsProcessor(10, eos_token_id = model.generation_config.eos_token_id),])
outputs = model.greedy_search(input_ids, logits_processor=logits_processor, return_dict_in_generate=True, output_scores=True, stopping_criteria=stopping_criteria)
# print(outputs[1])
transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)
print(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]

perplexity = 0
likelihood = 0

for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().numpy():.3f} | {np.exp(score.detach().numpy()):.2%}")
    likelihood += np.log(-1 * score.detach().numpy())
    perplexity += np.log(np.exp(score.detach().numpy()))

print(f"Length of the output: {generated_tokens.shape[1]}")
print(f"Perplexity: {np.exp((-1/ generated_tokens.shape[1]) * perplexity)}")
print(f"likelihood: {likelihood}")

["It might be possible to get a better understanding of the nature of the problem, but it's not clear how to do that.\n\nThe problem"]
|   651 |  get     | -3.341 | 3.54%
|   257 |  a       | -1.936 | 14.43%
|  1365 |  better  | -3.269 | 3.80%
|  4547 |  understanding | -1.486 | 22.63%
|   286 |  of      | -0.151 | 86.03%
|   262 |  the     | -1.247 | 28.73%
|  3450 |  nature  | -4.258 | 1.41%
|   286 |  of      | -0.083 | 92.02%
|   262 |  the     | -1.289 | 27.57%
|  1917 |  problem | -4.050 | 1.74%
|    11 | ,        | -1.583 | 20.54%
|   475 |  but     | -0.740 | 47.72%
|   340 |  it      | -1.707 | 18.14%
|   338 | 's       | -1.193 | 30.33%
|   407 |  not     | -1.242 | 28.88%
|  1598 |  clear   | -2.580 | 7.58%
|   703 |  how     | -1.511 | 22.07%
|   284 |  to      | -1.713 | 18.03%
|   466 |  do      | -1.116 | 32.77%
|   326 |  that    | -1.147 | 31.77%
|    13 | .        | -0.663 | 51.54%
|   198 | 
        | -1.139 | 32.01%
|   198 | 
        | -0.002 | 99.84%
|   464 | The

c:\Users\zebzi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1029: UserWarning: Use of index_put_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[indices] = tensor (Triggered internally at ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:634.)
  beam_indices[beam_indices_mask] = 0
c:\Users\zebzi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1029: UserWarning: Use of masked_fill_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[mask] = scalar (Triggered internally at ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1654.)
  beam_indices[beam_indices_mask] = 0


In [57]:
#Beam Search

outputs = model.generate(
    input_ids,
    max_new_tokens=20,
    num_beams=4,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True,
)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, outputs.beam_indices, normalize_logits=False
)
# If you sum the generated tokens' scores and apply the length penalty, you'll get the sequence scores.
# Tip: recomputing the scores is only guaranteed to match with `normalize_logits=False`. Depending on the
# use case, you might want to recompute it with `normalize_logits=True`.
output_length = input_length + np.sum(transition_scores.numpy() < 0, axis=1)
length_penalty = model.generation_config.length_penalty
reconstructed_scores = transition_scores.sum(axis=1) / (output_length**length_penalty)

print(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

# input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]

perplexity = 0
likelihood = 0

for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().numpy():.3f} | {np.exp(score.detach().numpy()):.2%}")
    likelihood += np.log(-1 * score.detach().numpy())
    perplexity += np.log(np.exp(score.detach().numpy()))

print(f"Length of the output: {generated_tokens.shape[1]}")
print(f"Perplexity: {np.exp((-1/ generated_tokens.shape[1]) * perplexity)}")
print(f"likelihood: {likelihood}")

["It might be possible to get a better idea of what is going on, but it's not going to be easy.\n", "It might be possible to get a better idea of what is going on, but it's not going to happen.\n\n", "It might be possible to get a better idea of what is going on, but it's not going to be easy. The", "It might be possible to get a better idea of what is going on, but it's not going to be easy. It"]
|   651 |  get     | -3.341 | 3.54%
|   257 |  a       | -1.936 | 14.43%
|  1365 |  better  | -3.269 | 3.80%
|  2126 |  idea    | -1.894 | 15.04%
|   286 |  of      | -0.236 | 79.01%
|   644 |  what    | -1.419 | 24.20%
|   318 |  is      | -3.010 | 4.93%
|  1016 |  going   | -1.086 | 33.77%
|   319 |  on      | -0.074 | 92.86%
|    11 | ,        | -2.350 | 9.54%
|   475 |  but     | -0.540 | 58.28%
|   340 |  it      | -1.867 | 15.46%
|   338 | 's       | -1.378 | 25.20%
|   407 |  not     | -1.351 | 25.90%
|  1016 |  going   | -2.077 | 12.53%
|   284 |  to      | -0.013 | 98.71%
|   307 |  

In [58]:
#Top_k Sampling

outputs = model.contrastive_search(input_ids, do_sample=True, max_length=20, stopping_criteria=stopping_criteria, return_dict_in_generate=True, output_scores=True)

print(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]

transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)

perplexity = 0
likelihood = 0

for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().numpy():.3f} | {np.exp(score.detach().numpy()):.2%}")
    likelihood += np.log(-1 * score.detach().numpy())
    perplexity += np.log(np.exp(score.detach().numpy()))

print(f"Length of the output: {generated_tokens.shape[1]}")
print(f"Perplexity: {np.exp((-1/ generated_tokens.shape[1]) * perplexity)}")
print(f"likelihood: {likelihood}")

["It might be possible to get a better understanding of the nature of the problem, but it's not clear how to do that.\n\nThe problem"]
|   651 |  get     | -3.341 | 3.54%
|   257 |  a       | -1.936 | 14.43%
|  1365 |  better  | -3.269 | 3.80%
|  4547 |  understanding | -1.486 | 22.63%
|   286 |  of      | -0.151 | 86.03%
|   262 |  the     | -1.247 | 28.73%
|  3450 |  nature  | -4.258 | 1.41%
|   286 |  of      | -0.083 | 92.02%
|   262 |  the     | -1.289 | 27.57%
|  1917 |  problem | -4.050 | 1.74%
|    11 | ,        | -1.583 | 20.54%
|   475 |  but     | -0.740 | 47.72%
|   340 |  it      | -1.707 | 18.14%
|   338 | 's       | -1.193 | 30.33%
|   407 |  not     | -1.242 | 28.88%
|  1598 |  clear   | -2.580 | 7.58%
|   703 |  how     | -1.511 | 22.07%
|   284 |  to      | -1.713 | 18.03%
|   466 |  do      | -1.116 | 32.77%
|   326 |  that    | -1.147 | 31.77%
|    13 | .        | -0.663 | 51.54%
|   198 | 
        | -1.139 | 32.01%
|   198 | 
        | -0.002 | 99.84%
|   464 | The

In [60]:
outputs = model.generate(input_ids, top_p = 4, do_sample=True, max_length=30, return_dict_in_generate=True, output_scores=True)
print(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]

transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)

perplexity = 0
likelihood = 0

for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().numpy():.3f} | {np.exp(score.detach().numpy()):.2%}")
    likelihood += np.log(-1 * score.detach().numpy())
    perplexity += np.log(np.exp(score.detach().numpy()))

print(f"Length of the output: {generated_tokens.shape[1]}")
print(f"Perplexity: {np.exp((-1/ generated_tokens.shape[1]) * perplexity)}")
print(f"likelihood: {likelihood}")

|  1486 |  design  | -4.740 | 0.87%
|   290 |  and     | -3.173 | 4.19%
|  1382 |  build   | -0.995 | 36.96%
|  3341 |  systems | -4.780 | 0.84%
|  1262 |  using   | -3.235 | 3.93%
|  4683 |  existing | -2.848 | 5.80%
|   513 |  3       | -5.035 | 0.65%
|    35 | D        | -0.109 | 89.72%
|  4981 |  models  | -2.218 | 10.89%
|    11 | ,        | -1.177 | 30.81%
|   290 |  and     | -3.198 | 4.09%
|  3863 |  maybe   | -4.084 | 1.68%
|   779 |  use     | -3.466 | 3.12%
|  4683 |  existing | -2.760 | 6.33%
|  3788 |  software | -3.622 | 2.67%
|   284 |  to      | -0.816 | 44.21%
|   787 |  make    | -2.445 | 8.67%
|   606 |  them    | -1.699 | 18.29%
|   517 |  more    | -1.629 | 19.61%
| 12846 |  flexible | -3.002 | 4.97%
|   290 |  and     | -1.523 | 21.81%
|  6068 |  adapt   | -2.681 | 6.85%
|   540 | able     | -0.018 | 98.19%
|    13 | .        | -0.542 | 58.16%
|   314 |  I       | -3.616 | 2.69%
Length of the output: 25
Perplexity: 12.635014748241938
likelihood: 13.84558696688466
